<font size = "6"> Project - Machine Learning and Computational Statistics
    
<font size = "5">Melina Moniaki - f3352321

In [ ]:
import scipy.io as sio
import pandas as pd
import numpy as np
import scipy.optimize 
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from scipy.optimize import minimize
from sklearn.linear_model import Lasso

In [ ]:
Pavia = sio.loadmat("C:\\Users\\melin\\Desktop\\Data Science\\Machine Learning and Computational Statistics\\Project\\PaviaU_cube.mat")
HSI = Pavia['X'] #Pavia HSI : 300x200x103
ends = sio.loadmat("C:\\Users\\melin\\Desktop\\Data Science\\Machine Learning and Computational Statistics\Project\\PaviaU_endmembers.mat") # Endmember's matrix: 103x9
endmembers = ends['endmembers']
fig = plt.figure()
plt.plot(endmembers)
plt.ylabel('Radiance values')
plt.xlabel('Spectral bands')
plt.title('9 Endmembers spectral signatures of Pavia University HSI')
plt.show()

In [ ]:
endmembers.shape
X=np.array(endmembers)

In [ ]:
X[:,1].shape

In [ ]:
#Perform unmixing for the pixels corresponding to nonzero labels
ground_truth= sio.loadmat("C:\\Users\\melin\\Desktop\\Data Science\\Machine Learning and Computational Statistics\\Project\\PaviaU_ground_truth.mat")
labels=ground_truth['y']
fig = plt.figure()
plt.imshow(HSI[:,:,10])
plt.title('RGB Visualization of the 10th band of Pavia University HSI')
plt.show()
# For the non-negative least squares  unmixing algorithm  you can use the nnls function, see the following link:
#https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.nnls.html#scipy.optimize.nnls

In [ ]:
print (HSI)
HSI.shape

<font size = "5"> **PART 1 - SPECTRAL UNMIXING**

<font size = "5"> **(a) Least squares (as it was presented in the class)**

Below, we take only the pixels with nonzero class label and put them on a new array with the name Y.(it consists of 12829 pixels).

In [ ]:
# In this project we take into consideration only the pixels with nonzero class label 
y=[]
for i in range (300):
    for j in range (200):
        if labels[i,j] != 0:
            y.append(HSI[i,j])
Y=np.array(y).T           

In [ ]:
print(Y)

In [ ]:
Y.shape

In [ ]:
X.shape

we perform least squares to calculate the values of theta and store them in a data frame called theta_est. We also calculate the reconstruction error for this method and then and derive the 9 abundance maps.

In [ ]:
# Perform least squares unmixing
XTX_inv = np.linalg.inv(np.dot(X.T, X))
theta_est = np.dot(XTX_inv, X.T).dot(Y)

theta_est_df=pd.DataFrame(theta_est)
theta_est_df

In [ ]:
# Initialize a list to store reconstruction errors for each pixel
reconstruction_errors = []

# Compute reconstruction errors for each pixel
for i in range(12829):
    # Calculate the estimated spectral signature using the estimated abundance
    y_est = np.dot(X, theta_est_df.iloc[:, i])

    # Compute the reconstruction error for the ith pixel
    error_i = np.square(np.linalg.norm(Y[:, i] - y_est, ord=2))  # Euclidean norm

    # Append the error to the list
    reconstruction_errors.append(error_i)

# Calculate the average reconstruction error
average_error = np.mean(reconstruction_errors)

print("Average Reconstruction Error:", average_error)

In [ ]:
labels.shape

Below we convert the data frame with the thetas (9x12829) to  a 3D array with dimensions 300x200x9 and we create 9 different 2D arrays, each corresponding to a different endmember/material. We do this to derive the corresponding 9 abundance maps (one for each endmember/material).

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array = np.zeros((300, 200, 9))

# Iterate over the pixels and fill in the values from the theta_est_df Data Frame
index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:  
            result_array[i,j,:] = theta_est_df.iloc[:,index]
            index += 1


In [ ]:
result_array.shape

In [ ]:
result_arrays = []

for k in range(9):
    result_arrays.append(result_array[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta = []
for i in range(9):
    df_a = pd.DataFrame(result_arrays[i])
    df_theta.append(df_a)

In [ ]:
for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()
    
    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

<font size = "5"> **(b) Least squares imposing the sum-to-one constraint**

we perform least squares to calculate the values of theta, this time imposing the sum-to-one constraint, and store them in a data frame called theta_est_constrained. We also calculate the reconstruction error for this method and derive the 9 abundance maps.

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

# Define the sum-to-one constraint function
def constraint(theta):
    return np.sum(theta) - 1.0

# Define the objective function for minimization (Euclidean norm)
def objective(theta):
    y_est = np.dot(X, theta)
    return np.linalg.norm(Y[:, i] - y_est, ord=2)  # Euclidean norm

# Initialize an array to store the results
theta_est_constrained = np.zeros((9, 12829))

# Iterate over all pixels
for i in range(12829):
    # Initialize the optimization with an equal distribution
    initial_guess = np.ones(9) / 9.0

    # Define the optimization problem with the sum-to-one constraint
    constraint_definition = {'type': 'eq', 'fun': constraint}
    optimization_result = minimize(objective, initial_guess, constraints=constraint_definition)

    # Store the optimized abundance vector
    theta_est_constrained[:, i] = optimization_result.x

# Calculate the reconstruction error
reconstruction_error_constrained = 0
for i in range(12829):
    y_est_constrained = np.dot(X, theta_est_constrained[:, i])
    reconstruction_error_constrained += np.square(np.linalg.norm(Y[:, i] - y_est_constrained, ord=2))

reconstruction_error_constrained /= 12829

print("The parameters θ1, θ2, ..., θ9 with sum-to-one constraint:\n", theta_est_constrained)
print("The Reconstruction Error with sum-to-one constraint is:", reconstruction_error_constrained)

In [ ]:
theta_est_constrained_df=pd.DataFrame(theta_est_constrained)
theta_est_constrained_df

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array_b = np.zeros((300, 200, 9))

# Iterate over the pixels and fill in the values from the theta_est_constrained array
index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0: 
            result_array_b[i,j,:] = theta_est_constrained_df.iloc[:,index]
            index += 1

In [ ]:
result_arrays_b = []

for k in range(9):
    result_arrays_b.append(result_array_b[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta_b = []
for i in range(9):
    df_b = pd.DataFrame(result_arrays_b[i])
    df_theta_b.append(df_b)

In [ ]:

for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()
    

    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta_b[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

<font size = "5"> **(c) Least squares imposing the non-negativity constraint on the entries of θ**

we perform least squares to calculate the values of theta, this time imposing the non-negativity constraint on the entries of θ, and store them in a data frame called theta_est_non_negativity. We also calculate the reconstruction error for this method and derive the 9 abundance maps.

In [ ]:
# Initialize an array to store the results
theta_est_non_negativity = np.zeros((9, 12829))

# Iterate over all pixels
for i in range(12829):
    # Perform non-negative least squares using nnls
    theta_non_negativity, _ = nnls(X, Y[:, i])

    # Store the non-negative least squares solution
    theta_est_non_negativity[:, i] = theta_non_negativity

# Calculate the reconstruction error
reconstruction_error_non_negativity = 0
for i in range(12829):
    y_est_non_negativity = np.dot(X, theta_est_non_negativity[:, i])
    reconstruction_error_non_negativity += np.square(np.linalg.norm(Y[:, i] - y_est_non_negativity, ord=2))

reconstruction_error_non_negativity /= 12829

print("The parameters θ1, θ2, ..., θ9 with non-negativity constraint using nnls:\n", theta_est_non_negativity)
print("The Reconstruction Error with non-negativity constraint using nnls is:", reconstruction_error_non_negativity)

In [ ]:
theta_est_non_negativity_df=pd.DataFrame(theta_est_non_negativity)
theta_est_non_negativity_df

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array_c = np.zeros((300, 200, 9))

index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:  
            result_array_c[i,j,:] = theta_est_non_negativity_df.iloc[:,index]
            index += 1

In [ ]:
result_arrays_c = []

for k in range(9):
    result_arrays_c.append(result_array_c[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta_c = []
for i in range(9):
    df_c = pd.DataFrame(result_arrays_c[i])
    df_theta_c.append(df_c)

In [ ]:

for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()
    

    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta_c[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

<font size = "5"> **(d) Least squares imposing both the non-negativity and the sum-to-one constraint on the entries of θ.**

we perform least squares to calculate the values of theta, this time imposing both the sum-to-one and the non-negativity constraints on the entries of θ, and store them in a data frame called theta_est_combined_constraints. We also calculate the reconstruction error for this method and derive the 9 abundance maps.

In [ ]:
# Define the sum-to-one constraint function
def sum_to_one_constraint(theta):
    return np.sum(theta) - 1.0

# Define the objective function for minimization (Euclidean norm)
def objective(theta):
    y_est = np.dot(X, theta)
    return np.linalg.norm(Y[:, i] - y_est, ord=2)  # Euclidean norm

# Initialize an array to store the results
theta_est_combined_constraints = np.zeros((9, 12829))

# Iterate over all pixels
for i in range(12829):
    # Perform non-negative least squares using nnls as an initial guess
    theta_non_negativity, _ = nnls(X, Y[:, i])

    # Define the optimization problem with both sum-to-one and non-negativity constraints
    constraint_definitions = [{'type': 'eq', 'fun': sum_to_one_constraint}]
    bounds = [(0, None)] * len(theta_non_negativity)  # Non-negativity constraint

    # Perform the optimization with both constraints
    optimization_result = minimize(objective, theta_non_negativity, constraints=constraint_definitions, bounds=bounds)

    # Store the optimized abundance vector
    theta_est_combined_constraints[:, i] = optimization_result.x

# Calculate the reconstruction error
reconstruction_error_combined_constraints = 0
for i in range(12829):
    y_est_combined_constraints = np.dot(X, theta_est_combined_constraints[:, i])
    reconstruction_error_combined_constraints += np.square(np.linalg.norm(Y[:, i] - y_est_combined_constraints, ord=2))

reconstruction_error_combined_constraints /= 12829

print("The parameters θ1, θ2, ..., θ9 with both constraints:\n", theta_est_combined_constraints)
print("The Reconstruction Error with both constraints is:", reconstruction_error_combined_constraints)

In [ ]:
theta_est_combined_constraints_df=pd.DataFrame(theta_est_combined_constraints)
theta_est_combined_constraints_df

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array_d = np.zeros((300, 200, 9))

# Iterate over the pixels and fill in the values from the theta_est array
index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0: 
            result_array_d[i,j,:] = theta_est_combined_constraints_df.iloc[:,index]
            index += 1

In [ ]:
result_arrays_d = []

for k in range(9):
    result_arrays_d.append(result_array_d[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta_d = []
for i in range(9):
    df_d = pd.DataFrame(result_arrays_d[i])
    df_theta_d.append(df_d)

In [ ]:
for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()

    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta_d[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

<font size = "5"> **(e) LASSO, i.e., impose sparsity on θ via 𝑙1 norm minimization.**

We did lasso for two values of alpha. alpha=0.1 and alpha = 1

alpha = 0.1

In [ ]:
from sklearn.linear_model import Lasso

# Initialize an array to store the results
theta_est_lasso = np.zeros((9, 12829))

# Set the regularization strength
alpha = 0.1 

# Iterate over all pixels
for i in range(12829):

   # Perform LASSO regularization with increased max_iter
    lasso = Lasso(alpha=alpha, max_iter=100000)
    lasso.fit(X, Y[:, i])

    # Store the optimized abundance vector
    theta_est_lasso[:, i] = lasso.coef_

# Calculate the reconstruction error
reconstruction_error_lasso = 0

for i in range(12829):
    y_est_lasso = np.dot(X, theta_est_lasso[:, i])
    reconstruction_error_lasso += np.square(np.linalg.norm(Y[:, i] - y_est_lasso, ord=2))

reconstruction_error_lasso /= 12829

print("The parameters θ1, θ2, ..., θ9 with LASSO:\n", theta_est_lasso)
print("The Reconstruction Error with LASSO is:", reconstruction_error_lasso)

In [ ]:
theta_est_lasso_df=pd.DataFrame(theta_est_lasso)
theta_est_lasso_df

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array_e = np.zeros((300, 200, 9))

# Iterate over the pixels and fill in the values from the theta_est_lasso array
index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0: 
            result_array_e[i,j,:] = theta_est_lasso_df.iloc[:,index]
            index += 1

In [ ]:
result_arrays_e = []

for k in range(9):
    result_arrays_e.append(result_array_e[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta_e = []
for i in range(9):
    df_e = pd.DataFrame(result_arrays_e[i])
    df_theta_e.append(df_e)

In [ ]:

for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()
    

    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta_e[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

alpha = 1

In [ ]:
# Initialize an array to store the results
theta_est_lasso = np.zeros((9, 12829))

# Set the regularization strength
alpha = 1 

# Iterate over all pixels
for i in range(12829):

   # Perform LASSO regularization with increased max_iter
    lasso = Lasso(alpha=alpha, max_iter=100000)
    lasso.fit(X, Y[:, i])

    # Store the optimized abundance vector
    theta_est_lasso[:, i] = lasso.coef_

# Calculate the reconstruction error
reconstruction_error_lasso = 0

for i in range(12829):
    y_est_lasso = np.dot(X, theta_est_lasso[:, i])
    reconstruction_error_lasso += np.square(np.linalg.norm(Y[:, i] - y_est_lasso, ord=2))

reconstruction_error_lasso /= 12829

print("The parameters θ1, θ2, ..., θ9 with LASSO:\n", theta_est_lasso)
print("The Reconstruction Error with LASSO is:", reconstruction_error_lasso)

In [ ]:
theta_est_lasso_df=pd.DataFrame(theta_est_combined_constraints)
theta_est_lasso_df

In [ ]:
# Create a 200x300x9 array filled with zeros
result_array_e = np.zeros((300, 200, 9))

# Iterate over the pixels and fill in the values from the theta_est array
index = 0
for i in range(300):
    for j in range(200):
        if labels[i,j] != 0:  
            result_array_e[i,j,:] = theta_est_lasso_df.iloc[:,index]
            index += 1

In [ ]:
result_arrays_e = []

for k in range(9):
    result_arrays_e.append(result_array_e[:, :, k])

# Now result_arrays contains 9 2D arrays, each corresponding to a different k value
df_theta_e = []
for i in range(9):
    df_e = pd.DataFrame(result_arrays_e[i])
    df_theta_e.append(df_e)

In [ ]:

for i in range(9):
    # Create the heatmap using Matplotlib's imshow function
    fig, ax = plt.subplots()
    
    mask = df_theta[i] == 0
    
    # Set the colormap (cmap) to 'viridis' and set_bad to white
    cmap = plt.get_cmap('viridis')
    cmap.set_bad(color='white')
    
    # Apply the mask to the data
    im = ax.imshow(np.ma.masked_array(df_theta_e[i], mask))
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Set axis labels
    ax.set_xlabel('X axis label')
    ax.set_ylabel('Y axis label')
    
    # Add title
    ax.set_title('Abundance Map {}'.format(i + 1))
    
    # Show the plot
    plt.show()

For lasso with alpha = 1 we obtain better results in the abundance maps (more distinct classes).

<font size = ""> **(B) Compare the results obtained from the above five methods (focusing on the abundance maps and the reconstruction error) and comment briefly on them (utilize the class information given in “PaviaU_ground_truth.mat”).**

In [ ]:
#Plot the ground truth map
labels=ground_truth['y']
fig = plt.figure()
plt.imshow(labels)
plt.title('Ground truth')
plt.show()

Based on the reference map, we can observe 9 distinct classes to which each pixel is assigned. Our goal was to determine the contribution percentage (abundance) of each pure material in the formation of a given pixel. This was achieved by calculating θi values for each pixel using different methods. In the resulting abundance maps, pixels with higher θi values appear greener, signifying a greater probability that a specific material is present in that pixel.

Among the five methods employed, the "Least Squares with both constraints" and "Least Squares imposing the non-negativity constraint," along with the "Lasso method for a =1 ", exhibit more pronounced regions. In these methods, pixels with green coloration are clearly defined and align well with the ground truth map. However, it's worth noting that these methods also show a higher reconstruction error compared to others, with the Lasso method having the highest reconstruction error value.

In the case of the first two methods, "Least Squares" and "Least Squares with sum to 1 constraint," many regions overlap, leading to a lack of discrete regions. This contrasts with the methods mentioned earlier, where distinct regions are more apparent.

<font size = "5"> **PART 2 - CLASSIFICATION**

In [ ]:
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics as m
from sklearn.model_selection import cross_val_score
import seaborn as sns

In [ ]:
 # Trainining set for classification 
Pavia_labels = sio.loadmat("C:\\Users\\melin\\Desktop\\Data Science\\Machine Learning and Computational Statistics\\Project\\classification_labels_Pavia.mat")
Training_Set = (np.reshape(Pavia_labels['training_set'],(200,300))).T
Test_Set = (np.reshape(Pavia_labels['test_set'],(200,300))).T
Operational_Set = (np.reshape(Pavia_labels['operational_set'],(200,300))).T
fig = plt.figure()
plt.imshow(Training_Set)
plt.title('Labels of the pixels of the training set')
plt.show()

In [ ]:
train_df=pd.DataFrame(Training_Set)
train_df

In [ ]:
test_df=pd.DataFrame(Test_Set)
test_df

In [ ]:
Operational_Set_df = pd.DataFrame(Operational_Set)
Operational_Set_df

In [ ]:
#Creation of X_train and y_train
X_train = []
y_train = []
for i in range(0,300):
    for j in range (0,200):
        if Training_Set[i,j] != 0 :
            X_train.append(HSI[i,j])
            y_train.append(Training_Set[i,j])
X_train=np.array(X_train)
y_train=np.array(y_train)
X_train.shape
y_train.shape

In [ ]:
#Creation of X_test, y_test
X_test = []
y_test = []
for i in range(0,300):
    for j in range (0,200):
         if Test_Set[i,j] != 0 :
            X_test.append(HSI[i,j])
            y_test.append(Test_Set[i,j])
X_test=np.array(X_test)
y_test=np.array(y_test)
# X_test
y_test

<font size = "4"> **Naive Bayes Classifier**

In [ ]:
from sklearn import metrics as m
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
# Build a Gaussian Classifier
model = GaussianNB()

# (i) Train it based on the training set performing 10-fold cross-validation
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
mean_validation_error_NB = np.mean(1 - scores)  # error is 1 - accuracy
std_validation_error_NB = np.std(1 - scores)

# Print the results
print(f"Estimated Validation Error (Mean): {mean_validation_error_NB:.2f}")
print(f"Estimated Validation Error (Std): { std_validation_error_NB:.2f}")

In [ ]:
# Model training
model.fit(X_train, y_train)
# Predict Output
y_pred_Bayes = model.predict(X_test)
res=m.classification_report(y_test, y_pred_Bayes)
print(res)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_mat_NB = confusion_matrix(y_test, y_pred_Bayes)

# Plot the confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_mat_NB, annot=True, cmap='Blues', fmt='d', ax=ax)
labels = ['1', '2', '3', '4', '5','6','7','8','9']
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)
plt.show()

In [ ]:
# Identify classes that are not well separated (if any)
poorly_separated_classes_NB = [i for i in range(confusion_mat_NB.shape[0]) if confusion_mat_NB[i, i] == 0]
print("Poorly Separated Classes:", poorly_separated_classes_NB)

# Compute success rate
success_rate_NB = np.sum(np.diag(confusion_mat_NB)) / np.sum(confusion_mat_NB)
print(f"Success Rate: {success_rate_NB:.2f}")

<font size = "4"> **K-nearest Neighbor Classifier**

Firstly, we are going to find the best value of k

In [ ]:
k_values = [i for i in range (1,20)]
scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, X_train, y_train, cv=10)
    scores.append(np.mean(score))
    
#Find the k value with the highest accuracy
best_k = k_values[np.argmax(scores)]
best_accuracy = scores[np.argmax(scores)]

print(f"The best k value is {best_k} with an accuracy of {best_accuracy:.4f}")    

Now we take the k value with the highest accuracy (k=9) 

In [ ]:
from sklearn.metrics import  accuracy_score

# Create a k-Nearest Neighbors Classifier (with the best k-value which is 9)
knn_classifier = KNeighborsClassifier(n_neighbors=9)  

# (i) Train it based on the training set performing 10-fold cross-validation
scores = cross_val_score(knn_classifier, X_train, y_train, cv=10, scoring='accuracy')
mean_validation_error_knn = np.mean(1 - scores)  # error is 1 - accuracy
std_validation_error_knn = np.std(1 - scores)

# Print the results
print(f"Estimated Validation Error (Mean): {mean_validation_error_knn:.2f}")
print(f"Estimated Validation Error (Std): {std_validation_error_knn:.2f}")

# (ii) Train on the whole training set and evaluate on the test set
knn_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_test = knn_classifier.predict(X_test)

# Predict Output
res=m.classification_report(y_test, y_pred_test)
print(res)

In [ ]:
confusion_mat_KNN = confusion_matrix(y_test, y_pred_test)

# Plot the confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_mat_KNN, annot=True, cmap='Blues', fmt='d', ax=ax)
labels = ['1', '2', '3', '4', '5','6','7','8','9']
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)
plt.show()

In [ ]:
# Identify classes that are not well separated (if any)
poorly_separated_classes_knn = [i for i in range(confusion_mat_KNN.shape[0]) if confusion_mat_KNN[i, i] == 0]

# Compute success rate
success_rate_knn = np.sum(np.diag(confusion_mat_KNN)) / np.sum(confusion_mat_KNN)

# Print the results
print("Poorly Separated Classes:", poorly_separated_classes_knn)
print(f"Success Rate: {success_rate_knn:.2f}")

<font size = "4"> **Minimum Euclidean Distance Classifier**

In [ ]:
# First we divide the data into 10 folds
folds_X = np.array_split(X_train, 10)
folds_Y = np.array_split(y_train, 10)

In [ ]:
# Implementing 10-fold cross-validation
error_list = []
accuracy_list = []

for i in range(10):
    # Using the current fold as the test set and the rest as the training set
    test_X_fold = folds_X[i]
    test_Y_fold = folds_Y[i]
    
    train_X_fold = np.concatenate(folds_X[:i] + folds_X[i+1:])
    train_Y_fold = np.concatenate(folds_Y[:i] + folds_Y[i+1:])
    
    # Calculating the mean of each class in the training set
    class_means = {}
    for label in np.unique(train_Y_fold):
        class_samples = train_X_fold[train_Y_fold == label]
        class_means[label] = np.mean(class_samples, axis=0)
        
    # Classifying the test set using the minimum Euclidean distance
    predictions = []
    for sample in test_X_fold:
        distances = []
        for label, mean in class_means.items():
            distance = np.linalg.norm(sample - mean)
            distances.append((distance, label))
        prediction = min(distances)[1]
        predictions.append(prediction)
        
    # Calculating misclassification error and accuracy on the test set
    error = np.mean(predictions != test_Y_fold)
    error_list.append(error)
    
    accuracy = np.mean(predictions == test_Y_fold)
    accuracy_list.append(accuracy)

# print results
print("Average Accuracy: %0.2f with standard deviation: %0.2f" % (np.mean(accuracy_list), np.std(accuracy_list)))
print("Average Error: %0.2f with standard deviation: %0.2f" % (np.mean(error_list), np.std(error_list)))

In [ ]:
# Calculating the mean of each class in the training set
class_means_dict = {}

for index, class_label in enumerate(np.unique(y_train)):
    class_samples_train = X_train[y_train == class_label]
    class_means_dict[class_label] = np.mean(class_samples_train, axis=0)

# For each test sample, compute its Euclidean distance to each class mean
predictions_list = []

for sample_index in range(len(X_test)):
    distances_list = []
    
    for class_label, mean_vector in class_means_dict.items():
        distance_to_mean = np.linalg.norm(X_test[sample_index] - mean_vector)
        distances_list.append((distance_to_mean, class_label))

    # Assign the test sample to the class with the smallest distance
    predicted_class = min(distances_list)[1]
    predictions_list.append(predicted_class)


In [ ]:
res=m.classification_report(y_test, predictions_list)
print(res)

In [ ]:
confusion_mat_EMD = confusion_matrix(y_test, predictions_list)

# Plot the confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_mat_EMD, annot=True, cmap='Blues', fmt='d', ax=ax)
labels = ['1', '2', '3', '4', '5','6','7','8','9']
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)
plt.show()

In [ ]:
# (i) Identify classes that are not well separated (if any)
poorly_separated_classes_EMD = [i for i in range(confusion_mat_EMD.shape[0]) if confusion_mat_EMD[i, i] == 0]

# (ii) Compute success rate 
success_rate_EMD = np.sum(np.diag(confusion_mat_EMD)) / np.sum(confusion_mat_EMD)

print("Poorly Separated Classes (EMD):", poorly_separated_classes_EMD)
print(f"Success Rate (EMD): {success_rate_EMD:.2f}")

<font size = '4'> **Bayesian classifier**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report

# Create a Linear Discriminant Analysis Classifier
lda_classifier = LinearDiscriminantAnalysis()

# (i) Train it based on the training set performing 10-fold cross-validation
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Use cross_val_score to perform k-fold cross-validation and obtain accuracy scores
scores = cross_val_score(lda_classifier, X_train, y_train, cv=kf, scoring='accuracy')
mean_validation_error_lda = 1 - np.mean(scores)  # error is 1 - accuracy
std_validation_error_lda = np.std(1 - scores)

# Print the results
print(f"Estimated Validation Error (Mean): {mean_validation_error_lda:.2f}")
print(f"Estimated Validation Error (Std): {std_validation_error_lda:.2f}")

# (ii) Train on the whole training set and evaluate on the test set
lda_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred_test_lda = lda_classifier.predict(X_test)

# Evaluate the classifier on the test set
print("Classification Report on Test Set:")
print(classification_report(y_test, y_pred_test_lda))

In [ ]:
# Create the confusion matrix
confusion_mat_lda = confusion_matrix(y_test, y_pred_test_lda)

# Plot the confusion matrix
ax = plt.subplot()
sns.heatmap(confusion_mat_lda, annot=True, cmap='Blues', fmt='d', ax=ax)
labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix (LDA)')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)
plt.show()

In [ ]:
# (i) Identify classes that are not well separated (if any) for LDA
poorly_separated_classes_lda = [i for i in range(confusion_mat_lda.shape[0]) if confusion_mat_lda[i, i] == 0]

# (ii) Compute success rate for LDA
success_rate_lda = np.sum(np.diag(confusion_mat_lda)) / np.sum(confusion_mat_lda)

# (iii) Print the results for LDA
print("Poorly Separated Classes (LDA):", poorly_separated_classes_lda)
print(f"Success Rate (LDA): {success_rate_lda:.2f}")

<font size = "4">Compare the results of the classifiers and comment on them

From the success rate of each classifier we can deduce that the best classifier is the KNN classifier (89% success rate) , followed by the Bayes classifier (88%) , then the Naive Bayes classifier (66%) and lastly the minimum euclidean distance classifier (56%).
From the confusion matrices of our two best classifiers (knn and Bayes classifiers) we can see that there is a confusion between classes 3 and 8. 
Based on the Bayes classifier 100 labels are missclasified as 8 (true label = 3) from total of 536. So the missclasification error for class 3 is 18.66%. Also, 74 labels are misslasified as 3 (true label=8) from total of 461. So the missclasification error for class 8 is 16%.
Based on knn classifier, 72 labels are missclasified as 8 from total of 536(true label=3). So the misclassification error for class 3 is 13.43%. Also, 85 labels are misclassified as 3 (true label= 8) from total of 461. So the missclasification error for class 8 is : 18.44 %.
Furthermore, there are instances of confusion among additional classes, although the degree of confusion is not as pronounced as that observed between classes 3 and 8.

<font size = "5"> **PART 3 - COMBINATION**

<font size = "4"> Comment briefly on the possible correlation of the results obtained from the spectral unmixing procedure with those obtained from classification.

We observe a noticeable correlation between the outcomes derived from the spectral unmixing procedure and those obtained through classification. Specifically, employing the best unmixing methods, such as Least Squares with both constraints and Non-Negativity Constraints, reveals distinctly separated classes, indicated by a prominent green coloration. Similarly, our top-performing classifiers, namely k-Nearest Neighbors (knn) and Bayes, exhibit nearly diagonal confusion matrices, implying accurate assignment of the majority of pixels to their true classes. In spectral unmixing, higher values of θi signify a greater likelihood of assigning a specific pixel to class i, mirroring the approach taken in the classification method. Both methods demonstrate effective performance. 
Additionally, our classification method reveals confusion between classes 3 and 8, a phenomenon corroborated by the abundance maps. Notably, some pixels with true label 3 are colored green in abundance map 8, and vice versa, underscoring the intricacies of class distinctions in both methodologies.